In [1]:
%run installations.ipynb

     |████████████████████████████████| 49.5 MB 9.9 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 4.6 MB 18.6 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 9.7 MB 11.3 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Start writing code here...

import open3d as o3d
import copy
import numpy as np
import time
import glob





def load_point_clouds(pcl_path="./rotary_room_scanner/rotary_rsrc/clouds3/", voxel_size=0.02, step_size=10):
    pcls = []

    list_of_pcls = glob.glob(pcl_path + '*.ply')
    list_of_pcls.sort()
    for pcl_filepath in list_of_pcls[::step_size]:
        pcl = o3d.io.read_point_cloud(pcl_filepath)
        pcl_down = pcl.voxel_down_sample(voxel_size=voxel_size)
        pcls.append(pcl_down)
    return pcls

# The following two functions are taken from http://www.open3d.org/docs/release/tutorial/pipelines/global_registration.html
def preprocess_point_cloud(voxel_size, pc1):
    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pc1.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pc1_fpfh = o3d.registration.compute_fpfh_feature(
        pc1,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))

    # return pc1 with estimated normal info & fpfh feature vectors (33 feat)
    return pc1, pc1_fpfh


def pairwise_registration(pc1, pc2, voxel_size=0.02):

    cl1, ind = pc1.remove_statistical_outlier(nb_neighbors=20, std_ratio=1.0)
    cl2, ind2 = pc2.remove_statistical_outlier(nb_neighbors=20, std_ratio=1.0)

    pc1N, pc1_fpfh = preprocess_point_cloud(voxel_size, cl1)
    pc2N, pc2_fpfh = preprocess_point_cloud(voxel_size, cl2)

    global_distance_threshold = voxel_size * 1.5

    success = False

    result = o3d.registration.registration_ransac_based_on_feature_matching(
        pc1N,
        pc2N,
        pc1_fpfh,
        pc2_fpfh,
        global_distance_threshold,
        o3d.registration.TransformationEstimationPointToPoint(False),
        8,
        [
            o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.registration.CorrespondenceCheckerBasedOnDistance(
                global_distance_threshold)
        ], o3d.registration.RANSACConvergenceCriteria(2000000,  # Maximum number of RANSAC iterations
                                                      250  # Maximum number of validation steps
                                                      )
    )

    threshold = 0.02

    reg_p2p = o3d.registration.registration_icp(
        pc1N,
        pc2N,
        threshold,
        result.transformation,  # Let's use the global transformation result as our starting point
        o3d.registration.TransformationEstimationPointToPoint()
    )

    print("registration fitness: ", reg_p2p.fitness)

    transformation_icp = reg_p2p.transformation

    information_icp = o3d.registration.get_information_matrix_from_point_clouds(
        pc1N, pc2N, global_distance_threshold,
        reg_p2p.transformation)
    return transformation_icp, information_icp




def multiway_registration(pcls):
    # Reference: http://www.open3d.org/docs/latest/tutorial/Advanced/multiway_registration.html
    #input: all the pointclouds
    pose_graph = o3d.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.registration.PoseGraphNode(odometry))
    n_pcds = len(pcls)


    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcls[source_id], pcls[target_id])
            print("Build o3d.pipelines.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.registration.PoseGraphNode(
                        np.linalg.inv(odometry)))
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=True))
    return pose_graph



cl_path="rotary/clouds3/"
voxel_size=0.02
step_size=10


pcds_down = load_point_clouds()
print(f"Trying to merge  {len(pcds_down)} point clouds")
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    pose_graph = multiway_registration(pcds_down)


print("Optimizing PoseGraph ...")
# Set options for global optimization of pose graph
option = o3d.registration.GlobalOptimizationOption(
    max_correspondence_distance=voxel_size*1.5,
    edge_prune_threshold=0.25,
    reference_node=0)

# with o3d.utility.VerbosityContextManager(
#         o3d.utility.VerbosityLevel.Debug) as cm:
#     o3d.registration.global_optimization(
#         pose_graph,
#         o3d.registration.GlobalOptimizationLevenbergMarquardt(),
#         o3d.registration.GlobalOptimizationConvergenceCriteria(),
#         option)
print("Entering global optim")
o3d.registration.global_optimization(
        pose_graph,
        o3d.registration.GlobalOptimizationLevenbergMarquardt(),
        o3d.registration.GlobalOptimizationConvergenceCriteria(),
        option)


print("Entering point cloud merging")

pcd_combined = o3d.geometry.PointCloud()

for point_id in range(len(pcds_down)):
    pcds_down[point_id].transform(pose_graph.nodes[point_id].pose)
    pcd_combined += pcds_down[point_id]
pcd_combined_down = pcd_combined.voxel_down_sample(voxel_size=voxel_size)
o3d.io.write_point_cloud(f"multiway_registration{time.time()}.ply", pcd_combined_down)
o3d.visualization.draw_geometries([pcd_combined_down])



[Open3D DEBUG] ICP Iteration #18: Fitness 0.0317, RMSE 0.0117
[Open3D DEBUG] ICP Iteration #19: Fitness 0.0318, RMSE 0.0117
[Open3D DEBUG] ICP Iteration #20: Fitness 0.0317, RMSE 0.0117
[Open3D DEBUG] ICP Iteration #21: Fitness 0.0316, RMSE 0.0117
[Open3D DEBUG] ICP Iteration #22: Fitness 0.0316, RMSE 0.0116
[Open3D DEBUG] ICP Iteration #23: Fitness 0.0315, RMSE 0.0116
[Open3D DEBUG] ICP Iteration #24: Fitness 0.0315, RMSE 0.0116
[Open3D DEBUG] ICP Iteration #25: Fitness 0.0314, RMSE 0.0116
[Open3D DEBUG] ICP Iteration #26: Fitness 0.0314, RMSE 0.0116
[Open3D DEBUG] ICP Iteration #27: Fitness 0.0315, RMSE 0.0116
[Open3D DEBUG] ICP Iteration #28: Fitness 0.0315, RMSE 0.0116
[Open3D DEBUG] ICP Iteration #29: Fitness 0.0314, RMSE 0.0115
registration fitness:  0.0313588850174216
Build o3d.pipelines.registration.PoseGraph
[Open3D DEBUG] Pointcloud down sampled from 17270 points to 17220 points.
[Open3D DEBUG] Pointcloud down sampled from 10367 points to 10294 points.
:: Estimate normal with

KeyboardInterrupt: 